In [1]:
import pandas as pd

from matplotlib import pyplot as plt

from sklearn.preprocessing import LabelEncoder

from keras import layers
from keras import models

In [2]:
dateColumnNames = ['contact_date','Glycemie_der_date','HbA1c_der_date','der_date_poids','der_date_taille','first_contact_date']

dfView = pd.read_csv('PatientsHTA.zip',nrows=1)
df = pd.read_csv('PatientsHTA.zip',engine='c',parse_dates=dateColumnNames)

# Suppression des lignes trop peu nombreuses

Nous souhaitons faire un apprentissage en utilisant la dimension temporelle comme filtre pour le CNN. Pour ça il faut donc que nous ayons plusieurs entrée. Avant de commencer à traîter les données nous supprimons donc toutes les lignes qui n'ont pas plusieurs entrées de ```person_id```. Nous choisissons arbitrairement que pour être utile à l'apprentissage, il faut au moins 10 entrées dans cette colonne

In [3]:
valueCounts = df.person_id.value_counts()
dfEnought = df[df.person_id.isin(valueCounts[valueCounts.values >= 10].index)]

# Suppression des colonnes innutiles

## Suppression de ```Age_now```

Nous pouvons supprimer la colonne ```Age_now``` car les données qu'elle contient sont identique à celles de la colonne ```year_of_birth```

In [4]:
dfWithoutAgeNow = dfEnought.drop('Age_now', axis='columns')

## Suppression de ```contact_id```

In [5]:
dfWithoutContactID = dfWithoutAgeNow.drop('contact_id',axis='columns')

## Suppression des noms de médicaments

In [6]:
dfGroupedByMoleculeLabel = dfWithoutContactID.groupby('product_atc_code')[['molecule_label','short_name','long_name','Classe','product_atc']].count()
dfGroupedByMoleculeLabel

,molecule_label,short_name,long_name,Classe,product_atc
product_atc_code,,,,,
C02AC06,2276,2276,2276,2276,2276
C03BX03,655,655,655,655,655
C03CA01,402,402,402,402,402
C03DA01,86,86,86,86,86
C03DA04,10,10,10,10,10
C03EA04,3,3,3,3,3
C03EB01,10,10,10,10,10
C07AB03,3380,3380,3380,3380,3380
C07AB04,34,34,34,34,34


Nous voyons que les différentes colonnes de noms de médicaments sont identiques, nous pouvons donc n'en garder qu'une seule. Nous choisirons de garder ```product_atc_code```

In [7]:
dropColumnNames = dfGroupedByMoleculeLabel.columns.to_list()
dfWithATCCode = dfWithoutContactID.drop(dropColumnNames, axis='columns')

In [8]:
dfWithATCCode

,person_id,specialty_label,contact_date,cip,dosage_1,dose_1,dose_2,product_atc_code,box,quantity,...,HbA1c_der_date,HbA1c_der_mesure,gender_code,Age_presc,year_of_birth,Poids,der_date_poids,Taille,der_date_taille,first_contact_date
8,27746613.0,Médecin généraliste,2013-05-15,3.400960e+12,300.0,16800.0,56.0,C09XA02,3.0,1.00,...,NaT,NaN,M,93.0,1920.0,75.0,2016-02-11,166.0,2014-06-05,2012-10-17
9,27746613.0,Médecin généraliste,2014-12-19,3.400960e+12,300.0,16800.0,56.0,C09XA02,4.0,1.00,...,NaT,NaN,M,94.0,1920.0,75.0,2016-02-11,166.0,2014-06-05,2012-10-17
12,27746613.0,Médecin généraliste,2013-06-14,3.400960e+12,300.0,16800.0,56.0,C09XA02,3.0,1.00,...,NaT,NaN,M,93.0,1920.0,75.0,2016-02-11,166.0,2014-06-05,2012-10-17
24,27746613.0,Médecin généraliste,2013-01-10,3.400960e+12,300.0,16800.0,56.0,C09XA02,3.0,1.00,...,NaT,NaN,M,93.0,1920.0,75.0,2016-02-11,166.0,2014-06-05,2012-10-17
32,27746613.0,Médecin généraliste,2013-12-30,3.400960e+12,300.0,16800.0,56.0,C09XA02,3.0,1.00,...,NaT,NaN,M,93.0,1920.0,75.0,2016-02-11,166.0,2014-06-05,2012-10-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50621,14936234.0,Médecin généraliste,2016-09-11,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,1.00,...,NaT,NaN,M,81.0,1935.0,63.8,2017-07-27,162.0,2011-11-07,2002-09-24
50622,14936234.0,Médecin généraliste,2016-11-16,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,1.00,...,NaT,NaN,M,81.0,1935.0,63.8,2017-07-27,162.0,2011-11-07,2002-09-24
50625,26636453.0,Médecin généraliste,2016-11-16,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,0.75,...,2017-10-03,5.60,M,79.0,1937.0,83.9,2018-02-21,NaN,NaT,2012-02-14
50627,18889430.0,Médecin généraliste,2013-02-01,3.400940e+12,500.0,15000.0,30.0,C03CA01,1.0,0.50,...,2017-05-15,8.01,M,85.0,1928.0,NaN,NaT,NaN,NaT,2007-12-02


## Suppression des colonnes ```'*der*'```

Les colonnes ```*der*``` contiennent la dernière données. Cette donnée peut être récupérée grâce à la date de la visite et à aux valeurs mesurées. Par exemple, il n'est pas nécessaire d'avoir une colonne ```der_date``` ou ```der_mesure```. Les données de ces deux types de colonnes peuvent être récupéré grâce à la ligne qui correspond à la dernière date de la mesure, que l'on peut trouver grâce à la colonne ```contact_date```

In [9]:
derColumnNames = []

for c in dfWithATCCode.columns:
    if ('der_date' in c) or ('der_mesure' in c):
        derColumnNames.append(c)

dfWithoutDer = dfWithATCCode.drop(derColumnNames,axis='columns')

# Traîtement des données

## Conversion des données

### Ajout du temps entre chaque visite (ce que l'on veut prédire)

In [10]:
wait_time = dfWithoutDer.contact_date - dfWithoutDer.first_contact_date
wait_time = wait_time.dt.total_seconds() / (24 * 3600)
dfWithoutDer['wait_time'] = wait_time

### Encodage des valeurs non numériques

In [11]:
specialtyEncoder = LabelEncoder()
ATCCodeEncoder = LabelEncoder()
frequencyLabelEncoder = LabelEncoder()
traitementAutresLabelEncoder = LabelEncoder()
traitementInsulineLabelEncoder = LabelEncoder()
genderEncoder = LabelEncoder()

dfWithoutDer.specialty_label = specialtyEncoder.fit_transform(dfWithoutDer.specialty_label)
dfWithoutDer.product_atc_code = ATCCodeEncoder.fit_transform(dfWithoutDer.product_atc_code)
dfWithoutDer.frequency_label = frequencyLabelEncoder.fit_transform(dfWithoutDer.frequency_label.astype(str))
dfWithoutDer.Traitement_Autres_A10_dep_201701 = traitementAutresLabelEncoder.fit_transform(dfWithoutDer.Traitement_Autres_A10_dep_201701.astype(str))
dfWithoutDer.Traitement_Insulines_dep_201701 = traitementInsulineLabelEncoder.fit_transform(dfWithoutDer.Traitement_Insulines_dep_201701.astype(str))
dfWithoutDer.gender_code = ATCCodeEncoder.fit_transform(dfWithoutDer.gender_code)


### Conversion en ```TimeSeries```

on définit simplement le nouvel index comme la colonne donnant l'intervalle de temps entre chaque visite

In [12]:
ts = dfWithoutDer.drop(['person_id','first_contact_date'],axis='columns').set_index(dfWithoutDer.contact_date).drop('contact_date',axis='columns')

# Prédiction

## Création des données d'entraînement/test

In [13]:
yColumNames = ['product_atc_code','wait_time']

xData, yData = (ts.drop(yColumNames,axis='columns').to_numpy().astype('float32'), ts.loc[:,yColumNames].to_numpy().astype('float32'))

In [14]:
trainUse = int(ts.shape[0] * 80 / 100)
testUse = ts.shape[0] - trainUse

xTrain, xTest, yTrain, yTest = (xData[:trainUse],xData[-testUse:],yData[:trainUse],yData[-testUse:])

xTrain = xTrain.reshape(xTrain.shape[0],1,1,xTrain.shape[1])
xTest = xTest.reshape(xTest.shape[0],1,1,xTest.shape[1])

In [15]:
xTrain

array([[[[2.00000e+00, 3.40096e+12, 3.00000e+02, ..., 1.92000e+03,
          7.50000e+01, 1.66000e+02]]],


       [[[2.00000e+00, 3.40096e+12, 3.00000e+02, ..., 1.92000e+03,
          7.50000e+01, 1.66000e+02]]],


       [[[2.00000e+00, 3.40096e+12, 3.00000e+02, ..., 1.92000e+03,
          7.50000e+01, 1.66000e+02]]],


       ...,


       [[[2.00000e+00, 3.40095e+12, 8.00000e+01, ..., 1.94000e+03,
          7.20000e+01, 1.71000e+02]]],


       [[[2.00000e+00, 3.40095e+12, 8.00000e+01, ..., 1.94300e+03,
          8.40000e+01,         nan]]],


       [[[2.00000e+00, 3.40095e+12, 8.00000e+01, ..., 1.94600e+03,
          7.60000e+01, 1.76000e+02]]]], dtype=float32)

## Création du modèle

In [17]:
(1,1,xData.shape[1])

(1, 1, 21)

In [44]:
model = models.Sequential()

model.add(layers.Conv2D(15,kernel_size=(3,xData.shape[1]),activation='relu',input_shape=(1,1,xData.shape[1])))
#model.add(layers.MaxPool2D((1,1)))
#model.add(layers.Flatten())
#model.add(layers.Dense(2,activation='softmax'))

ValueError: Negative dimension size caused by subtracting 21 from 1 for '{{node conv1d_1/conv1d/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv1d_1/conv1d/Reshape, conv1d_1/conv1d/ExpandDims_1)' with input shapes: [?,1,1,21], [1,21,21,15].

In [35]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 1, 1, 21)          462       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 21)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 21)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 44        
Total params: 506
Trainable params: 506
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [37]:
model.fit(xTrain,yTrain,epochs=5,validation_data=(xTest,yTest))

Epoch 1/5


InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [32,2] and labels shape [64]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-37-5672e3e6df33>:1) ]] [Op:__inference_train_function_1246]

Function call stack:
train_function
